In [1]:
###############################################################################
# This notebook provides some tools for better integration between the        #
# Pacific EMIS and OnlineSBA. It does the following                           #
#   - Retrieve SchoolID, School Name, etc. to automatically produce the       #
#     entity characteristics file used to load into OnlineSBA                 #
###############################################################################

# Core stuff
import os
import json

# Data stuff
import pandas as pd # Data analysis
import xlrd # excel 
import pyodbc # SQL DB

# Pretty printing stuff
from IPython.display import display, HTML
import pprint
pp = pprint.PrettyPrinter(indent=4)

# Initial setup
test = 'MISAT' # NMCT
country = 'RMI' # FSM
cwd = os.getcwd()

# Configuration
with open('config.json', 'r') as file:
     config = json.load(file)

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
# Load the relevant data from EMIS database

# Establish a database server connection
conn = """
    Driver={{ODBC Driver 17 for SQL Server}};
    Server={},{};
    Database={};
    authentication=SqlPassword;UID={};PWD={};
    TrustServerCertificate=yes;
    autocommit=True
    """.format(config['server_ip'], config['server_port'], config['database'], config['uid'], config['pwd'])

sql_conn = pyodbc.connect(conn)

query = """
SELECT [schNo] AS SCHOOLID
      ,[schName] AS SCHOOLNAME
	  ,I.iName AS ISLAND
	  ,D.dName AS DISTRICT
	  ,A.authName AS AUTHORITY
	  ,AT.codeDescription AS AUTHORITYTYPE
	  ,AG.codeDescription AS URBAN
      ,[schClosed] AS CLOSED
      ,[schCloseReason] AS CLOSEDREASON
  FROM [dbo].[Schools] S
  INNER JOIN Islands I ON S.iCode = I.iCode
  INNER JOIN Districts D ON I.iGroup = D.dID
  INNER JOIN Authorities A ON S.schAuth = A.authCode
  INNER JOIN lkpAuthorityType AT ON A.authType = AT.codeCode
  INNER JOIN lkpAuthorityGovt AG ON AT.codeGroup = AG.codeCode
"""

query_ethnicity = """SELECT [codeDescription] AS ETHNICITY FROM [dbo].[lkpEthnicity]"""
                          
df_schools_x = pd.read_sql(query, sql_conn)
display(df_schools_x)
df_ethnicities = pd.read_sql(query_ethnicity, sql_conn)
display(df_ethnicities)

,SCHOOLID,SCHOOLNAME,ISLAND,DISTRICT,AUTHORITY,AUTHORITYTYPE,URBAN,CLOSED,CLOSEDREASON
0,AIL100,Airok Elementary School,Aerok,Ailinglaplap Atoll,Public School System,National,Public,0,None
1,AIL101,Buoj Elementary School,Buoj,Ailinglaplap Atoll,Public School System,National,Public,0,None
2,AIL102,Enewa Elementary School,Enewa,Ailinglaplap Atoll,Public School System,National,Public,0,None
3,AIL103,Jah Elementary School,Jah,Ailinglaplap Atoll,Public School System,National,Public,0,None
4,AIL104,Jeh Elementary School,Jeh,Ailinglaplap Atoll,Public School System,National,Public,0,None
...,...,...,...,...,...,...,...,...,...
109,UTR101,Utrik Elementary School,Utrik,Utirik Atoll,Public School System,National,Public,0,None
110,WOT101,Wotho Elementary School,Wotho,Wotho Atoll,Public School System,National,Public,0,None
111,WTH101,Northern Islands High School,Wotje,Wotje Atoll,Public School System,National,Public,0,None
112,WTH102,Wodmej Elementary School,Wodmej,Wotje Atoll,Public School System,National,Public,0,None


,ETHNICITY
0,Marshallese
1,Carolinian
2,Caucasian
3,Chamorro
4,Chinese
5,Chuukese
6,Fijian
7,Filipino
8,Kanak
9,Kwaio


In [4]:
df_schools = df_schools_x.copy()
df_schools = df_schools[['SCHOOLID','SCHOOLNAME','ISLAND','DISTRICT','URBAN']]
display(df_schools)

s_islands = df_schools['ISLAND'].drop_duplicates().reset_index(drop=True)
s_islands.rename('UNIQUEISLANDNAME', inplace=True)
display(s_islands)

s_rubric_levels = pd.Series(data=['Beginning','Developing','Proficient','Advanced'], name='RUBRICLEVELS')
display(s_rubric_levels)

s_strand_layers = pd.Series(data=['Indicator','Benchmark','Standard'], name='STRANDLAYERS')
display(s_strand_layers)

s_subject = pd.Series(data=['B:Reading (Marshallese)','M:Math','N:Math Form B',
                            'E:English','H:High School Entrance','S:Science'], name='SUBJECT')
display(s_subject)


,SCHOOLID,SCHOOLNAME,ISLAND,DISTRICT,URBAN
0,AIL100,Airok Elementary School,Aerok,Ailinglaplap Atoll,Public
1,AIL101,Buoj Elementary School,Buoj,Ailinglaplap Atoll,Public
2,AIL102,Enewa Elementary School,Enewa,Ailinglaplap Atoll,Public
3,AIL103,Jah Elementary School,Jah,Ailinglaplap Atoll,Public
4,AIL104,Jeh Elementary School,Jeh,Ailinglaplap Atoll,Public
...,...,...,...,...,...
109,UTR101,Utrik Elementary School,Utrik,Utirik Atoll,Public
110,WOT101,Wotho Elementary School,Wotho,Wotho Atoll,Public
111,WTH101,Northern Islands High School,Wotje,Wotje Atoll,Public
112,WTH102,Wodmej Elementary School,Wodmej,Wotje Atoll,Public


0      Aerok
1       Buoj
2      Enewa
3        Jah
4        Jeh
       ...  
71      Ujae
72     Utrik
73     Wotho
74     Wotje
75    Wodmej
Name: UNIQUEISLANDNAME, Length: 76, dtype: object

0     Beginning
1    Developing
2    Proficient
3      Advanced
Name: RUBRICLEVELS, dtype: object

0    Indicator
1    Benchmark
2     Standard
Name: STRANDLAYERS, dtype: object

0    B:Reading (Marshallese)
1                     M:Math
2              N:Math Form B
3                  E:English
4     H:High School Entrance
5                  S:Science
Name: SUBJECT, dtype: object

In [6]:
# Write various DataFrame into Excel to examine (testing)
#s = 'data/' + country + '/entity-characteristics-test.xlsx'

# Write various DataFrame into Excel (real one)
s = 'data/' + country + '/onlinesba-load-files-csv/'+ country +'-entity-characteristics.xlsx'

filename = os.path.join(cwd, s)
with pd.ExcelWriter(filename) as writer:
    # add DataFrames you want to write to Excel here
    df_schools.to_excel(writer, index=False, sheet_name='RMIEntityCharacteristics', engine='openpyxl')
    s_islands.to_excel(writer, index=False, sheet_name='RMIEntityCharacteristics', startcol=5, engine='openpyxl')
    df_ethnicities.to_excel(writer, index=False, sheet_name='RMIEntityCharacteristics', startcol=6, engine='openpyxl')
    s_rubric_levels.to_excel(writer, index=False, sheet_name='RMIEntityCharacteristics', startcol=7, engine='openpyxl')
    s_strand_layers.to_excel(writer, index=False, sheet_name='RMIEntityCharacteristics', startcol=8, engine='openpyxl')
    s_subject.to_excel(writer, index=False, sheet_name='RMIEntityCharacteristics', startcol=9, engine='openpyxl')